In [1]:
# https://docs.astral.sh/uv/getting-started/installation/
!uv add google-api-python-client pandas python-dotenv pendulum

      ▄▄▀▀▄▄      
    ▄▀▀▀▀▀▀▀▀▄▀▀▄ 
    ▀▀▀▀▀▀▀▀▀▀▀▀▄ 
▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄
  ▀▀   ▀▀▀▀▀▀▀▀▀▀▀
       ▄▀▀▀▀▀▀▀▀  
       ▀▀▀▀▀▀▀    
         ▀▀▀      

Resolved 128 packages in 0.49ms
Audited 124 packages in 0.92ms


In [2]:
import os
import pendulum
from dotenv import load_dotenv
from googleapiclient.discovery import build
import pandas as pd

load_dotenv()

True

In [3]:
API_KEY = os.getenv("YT_DATA_API")
youtube = build(serviceName="youtube", version="v3", developerKey=API_KEY)

In [4]:
def get_seed_videos(query: str, max_results=10, pages=1):
    video_data = []
    page_token = None

    for _ in range(pages):
        req = youtube.search().list(
            q=query,
            part="snippet",
            type="video",
            maxResults=max_results,
            order="relevance",
            pageToken=page_token,
        )

        res = req.execute()

        for item in res.get("items", []):
            video_data.append(
                {
                    "video_id": item["id"]["videoId"],
                    "title": item["snippet"]["title"],
                    "description": item["snippet"]["description"],
                    "channel_id": item["snippet"]["channelId"],
                    "channel_title": item["snippet"]["channelTitle"],
                    "publish_date": item["snippet"]["publishedAt"],
                }
            )

        page_token = res.get("nextPageToken")
        if not page_token:
            break
        time.sleep(0.5)


    return video_data


In [5]:
topics = [
    "Zenless Zone Zero guide",
    "Zenless Zone Zero character showcase",
    "Zenless Zone Zero deadly assualt",
    "Zenless Zone Zero shiyu defence",
    "Zenless Zone Zero tier list",
    "Zenless Zone Zero news",
    "Zenless Zone Zero gameplay",
    "Zenless Zone Zero cosplay"
]
# comment for production
topics=topics[:1]

In [12]:
videos=[]
for query in topics:
    videos.extend(
        get_seed_videos(query,max_results=10)
    )

In [13]:
df=pd.DataFrame(videos)
print(df.shape)
df.head()

(10, 6)


,video_id,title,description,channel_id,channel_title,publish_date
0,kR626gjg0MQ,Zenless Zone Zero Beginner&#39;s Guide - 20 Ti...,Zenless Zone Zero is here! Here are 20 tips an...,UCHMnsOPDgrjWvjHXZp-xvqg,IWinToLose Gaming,2024-07-04T21:50:14Z
1,poJkFK3_M_0,How to get 180 FREE pulls in Zenless Zone Zero...,,UCKy1dAqELo0zrOtPkf0eTMw,IGN,2024-07-06T00:19:44Z
2,6_znQMRcIxg,"If I Started Zenless Zone Zero In 2025, Here&#...",This beginner and new player friendly guide is...,UCSvjKqgK2kY4W0i2T5sTZBg,Rhinne,2025-08-01T16:16:54Z
3,vD2hbBykyVM,A COMPLETE Guide to Combat Basics in Zenless Z...,"In this complete guide, I go through all the c...",UCDbRoMWfEYzvZ9RlKeSH1Yw,Guoba Uncertified,2024-07-08T14:45:36Z
4,uAxjI2RKHFw,13 TIPS ALL ZENLESS ZONE ZERO PLAYERS SHOULD K...,Zenless Zone Zero version 2.0 is coming NEXT w...,UCvfKU8LUmdPRTt42kSktkWQ,Shanaha,2025-05-29T01:10:00Z


In [ ]:
data_path="../data"
if not os.path.exists(data_path):
    os.mkdir(data_path)
seed_videos_df.to_csv(f"{data_path}/dim_videos.csv", index=False)